In [1]:
#import useful libraries 
import pandas as pd
import json
%matplotlib inline 


##link to the dataset
## https://www.cs.cmu.edu/~ark/personas/

In [2]:
#create path directory
DATA_FOLDER = 'MovieSummaries/'

CHARACTER_DATASET = DATA_FOLDER+"character.metadata.tsv"
MOVIE_DATASET = DATA_FOLDER+"movie.metadata.tsv"
NAME_CLUSTER_DATASET = DATA_FOLDER+"name.clusters.txt"
PLOT_SUMMARY_DATASET = DATA_FOLDER+"plot_summaries.txt"
TROPES_CLUSTER_DATASET = DATA_FOLDER+"tvtropes.clusters.txt"

## Load the different datasets

In [12]:
#character dataset
df_character = pd.read_csv(CHARACTER_DATASET, sep='\t', header=None)
column_names_character = ['Wikipedia movie ID', 'Freebase movie ID', 'Release date', 'Character name', 'Birth date', 'Gender', \
                          'height [m]', 'Ethnicity (Freebase ID)', 'Actor name', 'Age at movie release', \
                          'Freebase character/actor map ID', 'Freebase character ID', 'Freebase actor ID']
df_character.columns = column_names_character
display(df_character)

,Wikipedia movie ID,Freebase movie ID,Release date,Character name,Birth date,Gender,height [m],Ethnicity (Freebase ID),Actor name,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450667,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm


In [11]:
#movie dataset
df_movie = pd.read_csv(MOVIE_DATASET, sep='\t', header=None)
column_names_movie = ['Wikipedia ID', 'Freebase ID', 'Name', 'Release date', 'Box office data', 'Runtime', \
                 'Languages (Freebase ID:name tuples)', 'Countries (Freebase ID:name tuples)', \
                 'genres (Freebase ID:name tuples)']
df_movie.columns = column_names_movie
display(df_movie)

,Wikipedia ID,Freebase ID,Name,Release date,Box office data,Runtime,Languages (Freebase ID:name tuples),Countries (Freebase ID:name tuples),genres (Freebase ID:name tuples)
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [10]:
#name cluster dataset
df_name_cluster = pd.read_csv(NAME_CLUSTER_DATASET, delimiter=r'\s+/', header=None, engine='python')
column_name_cluster = ['Actor name', 'Freebase character/actor map ID']

df_name_cluster.columns = column_name_cluster
display(df_name_cluster)

,Actor name,Freebase character/actor map ID
0,Stuart Little,m/0k3w9c
1,Stuart Little,m/0k3wcx
2,Stuart Little,m/0k3wbn
3,John Doe,m/0jyg35
4,John Doe,m/0k2_zn
...,...,...
2661,John Rolfe,m/0k5_ql
2662,John Rolfe,m/02vd6vs
2663,Elizabeth Swann,m/0k1xvz
2664,Elizabeth Swann,m/0k1x_d


In [9]:
# Read the text file line by line
with open(PLOT_SUMMARY_DATASET, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    
# Create empty lists to store extracted data
column1 = []
column2 = []

# Loop through lines and extract data
for line in lines:
    parts = line.split(' ', 1)  # Split on the first space
    if len(parts) == 2:
        part1 = parts[0].strip()
        part2 = parts[1].strip()
        if part1.isdigit():  # Check if the first part is a number
            column1.append(part1)
            column2.append(part2)

df_summary = pd.DataFrame({'Wikipedia ID movie': column1, 'Summary': column2})

display(df_summary)

,Wikipedia ID movie,Summary
0,529276,"On the island of Amity, sheriff Martin Brody, ..."
1,33942920,"In 1906 London, orphan Peter Pan is one of a ..."
2,4995581,While the novel is told by a third-person narr...
3,11250635,The story starts as one of the robots flies in...
4,11102452,"Down a dirty alleyway, Butch the alley cat is ..."
...,...,...
1104,9957782,"Tati Benitez , is a young man who lives in the..."
1105,4107170,"The film begins with a historical overview, st..."
1106,22734574,"By the Bank of the River Nile: 3,300 years ago..."
1107,31009222,"In Pasadena, high school students Costa and J...."


In [7]:
# Read the text file line by line
with open(TROPES_CLUSTER_DATASET, 'r') as file:
    lines = file.readlines()
    
# Create empty lists to store extracted data
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

# Loop through lines and extract data
for line in lines:
    parts = line.strip().split('\t')
    if len(parts) == 2:
        column1.append(parts[0])  # Extract the first part
        data = json.loads(parts[1])  # Parse the JSON string in the second part
        column2.append(data.get('char', ''))  # Extract 'char' value
        column3.append(data.get('movie', ''))  # Extract 'movie' value
        column4.append(data.get('id', ''))  # Extract 'id' value
        column5.append(data.get('actor', ''))  # Extract 'actor' value
        
df_tropes_cluster = pd.DataFrame({'Character types': column1, 'Char': column2, 'Movie': column3, 'Freebase character/actor map ID': column4, 'Actor': column5})

display(df_tropes_cluster)

,Character types,Char,Movie,Freebase character/actor map ID,Actor
0,absent_minded_professor,Professor Philip Brainard,Flubber,/m/0jy9q0,Robin Williams
1,absent_minded_professor,Professor Keenbean,Richie Rich,/m/02vchl3,Michael McShane
2,absent_minded_professor,Dr. Reinhardt Lane,The Shadow,/m/0k6fkc,Ian McKellen
3,absent_minded_professor,Dr. Harold Medford,Them!,/m/0k6_br,Edmund Gwenn
4,absent_minded_professor,Daniel Jackson,Stargate,/m/0k3rhh,James Spader


## Process data and first analysis

In [24]:
##percentage of missing values 

percentage_missing_values = (df_movie['Release date'].isna().sum()/len(df_movie['Release date']))*100
print(f"The percentage of missing values is {format(percentage_missing_values, '.3f')} %")

The percentage of missing values is 8.444 %
